In [2]:
import numpy as np
import os
from processing.data_manager import SHREDDataManager
from models import models
import torch
import numpy as np
from misc.example_helper import *

In [3]:
sample = np.load('data14fields_npz/EfX_3D.npz')
sample = sample[sample.files[0]]
sensor_perimeter_walk_coordinates = perimeter_walk(height = sample.shape[0], width = sample.shape[1], timesteps = sample.shape[2], clockwise=True)

In [4]:
import os

# Initialize SHREDDataManager
manager = SHREDDataManager(lags=20, train_size=0.7, val_size=0.15, test_size=0.15)

# Folder containing .npz files
input_folder = "data14fields_npz"

# Process each .npz file
for filename in os.listdir(input_folder):
    if filename.endswith(".npz") and filename != 'ne_3D.npz':
        file_path = os.path.join(input_folder, filename)

        # Add the file to the SHREDDataManager
        manager.add(
            file_path=file_path,
            compression=20,
            scaling="minmax",
            time=np.arange(0, 2000)  # Assuming all files have 2000 timesteps
        )
        print(f"Processed: {filename}")

filename = 'ne_3D.npz'
file_path = os.path.join(input_folder, filename)

manager.add(
    file_path=file_path,
    random_sensors=3,
    stationary_sensors=(7, 9),  # Example coordinates
    mobile_sensors=sensor_perimeter_walk_coordinates,
    compression=20,
    scaling="minmax",
    time=np.arange(0, 2000)  # Assuming all files have 2000 timesteps
)

In [5]:
train_set, val_set, test_set = manager.preprocess()

In [8]:
train_set.reconstructor.X.shape

torch.Size([1386, 21, 5])

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
shred = models.SHRED(train_set.reconstructor.X.shape[-1], train_set.reconstructor.Y.shape[-1], hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
validation_errors = models.fit(shred, train_set.reconstructor, val_set.reconstructor, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=40)

Training epoch 1
Error tensor(0.2846)
Training epoch 20
Error tensor(0.2804)


KeyboardInterrupt: 

In [ ]:
prediction = shred(test_set.reconstructor.X).detach().cpu().numpy()
prediction.shape